In [5]:
import os, sys

from datetime import datetime, timedelta
import time

from inlayr.provider import get_provider

### Init

In [6]:
provider = get_provider(
    chain = "evm",
    chain_params = {
        "wallet_pk": open(os.path.join(os.environ.get("inlayr_root"), r"keys\ethereum_wallet")).read(),
    },
    
    rpc = "ankr", 
    rpc_params = {
        "chain_id": 1,
        "api_key": open(os.path.join(os.environ.get("inlayr_root"), r"keys\ankr-api-key")).read(),
    }, 
    
    aggregator = "zeroex",
    aggregator_params = {
        "headers": {
            "0x-version": "v2",
            "0x-api-key": open(os.path.join(os.environ.get("inlayr_root"), r"keys\0x-api-key")).read(),
        }
    }
)

### Inputs

In [7]:
source_token = "0xdac17f958d2ee523a2206206994597c13d831ec7" # USDT mint address
destination_token = "0xf57e7e7c23978c3caec3c3548e3d615c346e79ff" 

source_amount = int(150 * 1e6)
slippage = int(0.05 * 10000) # Convert to bps

### Set allowance

In [13]:
approval_transaction = provider.get_approval(
    source_token = source_token,
    source_amount = source_amount
)

Allowance is sufficient, no approval needed.


In [14]:
if approval_transaction.transaction is not None:
    approval_executed = provider.execute_trx(
        transaction = approval_transaction
    )

In [15]:
if approval_transaction.transaction is not None:
    display(
        provider.trx_status(
            signature = approval_executed
        )
    )

### Execute swap

In [16]:
quote = provider.get_quote(
    source_token = source_token, 
    source_amount = source_amount, 
    destination_token = destination_token, 
    slippage = slippage
)

print ("max gas fee:", (1.0e-18) * int(quote.quote['transaction']['gas']) * int(quote.quote['transaction']['gasPrice']), "ETH")

max gas fee: 0.00018370895957946 ETH


In [17]:
swap_transaction = provider.get_swap(
    quote = quote
)

In [18]:
swap_executed = provider.execute_trx(
    transaction = swap_transaction
)

In [ ]:
provider.trx_status(
    signature = swap_executed
)

### Check balance

In [8]:
token_position = provider.get_balance(
    source_token = source_token
)

token_position

Balance(source='EVM', amount=89727)